In [1]:
%pylab inline

In [2]:
import sys, os
top_dir = '/Users/rjl/student_workshop2'
codes_dir = os.path.join(top_dir, 'Codes')
sys.path.append(codes_dir)

In [3]:
CCmap = imread('/Users/rjl/student_workshop2/maps/CCmap-small.png')
extent = (235.79781, 235.82087, 41.739671,41.762726)   #small region

In [4]:
def plot_CCmap():
    fig = figure(figsize=(6,6))
    ax = axes()
    imshow(CCmap,extent=extent)
    CClatitude = 41.75  # to rescale longitude
    ax.set_aspect(1. / cos(pi*CClatitude/180.)) 
    ax.ticklabel_format(format='plain',useOffset=False)

In [5]:
fixed_grid_file = os.path.join(top_dir, 'Datafiles/fixedgrid_xyB_small.npy')
d=load(fixed_grid_file)
x=d[:,0]
y=d[:,1]
B=d[:,2]
topo = reshape(B, (250,250), order='F')
X = reshape(x, (250,250), order='F')
Y = reshape(y, (250,250), order='F')

In [6]:
realizations_dir = os.path.join(top_dir, 'Realizations')

In [10]:
zeta1=[0.0,.1,.2,.3,.4,.5,.6,.7,.8,.9,1.0,1.1,1.2,1.3,1.4,1.5,1.6,1.7,1.8]
zeta2=[1.9,2.0,2.5,3.0,3.5,4.0,4.5,5.0,5.5,6.0]
zeta3=[7.0,8.0,9.0,10.0,11.0,12.0]
zeta=array(zeta1+zeta2+zeta3)
nzeta = len(zeta)
print '%i exceedance values, zeta = \n %s' % (nzeta,zeta)


In [19]:
def combine_prob(p1,p2):
    return 1. - (1-p1)*(1-p2)

In [30]:
prob = zeros((250,250,nzeta))  # initialize to zero

In [31]:
event = 'AASZe01r01'
event_dir = os.path.join(realizations_dir, event)
hmax_file = os.path.join(event_dir, 'tide+077', 'h_eta_small.npy')
hmax = load(hmax_file)
Hmax = hmax.reshape((250,250),order='F')
p_event = 0.1

In [35]:
for k in range(nzeta):
    Pk = prob[:,:,k]  # probabilities at all points for one exceedance value
    prob[:,:,k] = where(Hmax > zeta[k], combine_prob(p_event,Pk), Pk)

In [36]:
prob.max()

In [39]:
k = 0
plot_CCmap()
prob_clines = [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1]
contourf(X,Y,prob[:,:,k],prob_clines,alpha = 0.6)
title("Annual probability of flooding above %g meters" % zeta[k])
colorbar()

In [27]:
fig = figure(figsize=(6,6))
ax = axes()

H_clines = [0.01, 0.5, 1., 1.5, 2., 2.5, 3]
contourf(X,Y,Hmax,H_clines)
colorbar()

contour(X,Y,topo,[0.], colors='r')
topo_clines = linspace(2,20,10)
contour(X,Y,topo,topo_clines, colors='k')

CClatitude = 41.75  # to rescale longitude
ax.set_aspect(1. / cos(pi*CClatitude/180.)) 
ax.ticklabel_format(format='plain',useOffset=False)